In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\Chatbot\\research'

In [3]:
%cd ../

d:\Chatbot


c:\Users\klnis\anaconda3\envs\medchat\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
%pwd

'd:\\Chatbot'

In [5]:
from langchain.document_loaders import  PyPDFLoader, DirectoryLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def pdf_loader(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    pdf = loader.load()
    return pdf

In [7]:
def csv_loader(data):
    loader = DirectoryLoader(data, glob="*.csv", loader_cls=CSVLoader)
    csv = loader.load()
    return csv

In [8]:
pdf = pdf_loader("data")
csv = csv_loader("data")

In [16]:
#pdf

In [15]:
#csv

In [9]:
extracted_data = pdf + csv

In [14]:
#extracted_data

In [10]:
def text_splitts(data):
    text_splitters = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunk = text_splitters.split_documents(data)
    return text_chunk

In [11]:
chunk_text = text_splitts(extracted_data)

In [12]:
len(chunk_text)

835758

In [21]:
#chunk_text

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = download_embeddings()

C:\Users\klnis\AppData\Local\Temp\ipykernel_18996\2372466182.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\klnis\anaconda3\envs\medchat\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
query_result = embeddings.embed_query("hello world")

In [9]:
len(query_result)

384

In [28]:
#query_result

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
import os

In [12]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [13]:
index_name = "medicalbot"

In [21]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

In [14]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [15]:
from langchain_pinecone import PineconeVectorStore

In [24]:
doc_search = PineconeVectorStore.from_documents(
    documents=chunk_text,
    index_name=index_name,
    embedding=embeddings,
)

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [16]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [17]:
docsearch

In [18]:
retriver = docsearch.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [19]:
retriver_doc = retriver.invoke("what is acne")

In [20]:
retriver_doc

[Document(id='1ec58b68-8e08-4c09-93f7-bde85b94dc9b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='f08a03db-a896-4592-92ea-ff7fc10bd36a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [21]:
from langchain_groq import ChatGroq

In [22]:
llm = ChatGroq(
    model='llama-3.3-70b-versatile',
    temperature=0.4,
    max_tokens=500,
)

In [23]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever, ConversationalRetrievalChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [24]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

In [25]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [26]:
system_template = (
    "You are an assistant for question-answering tasks. "
    "Use the retrieved documents as context but do not make up information. "
    "If you don't know the answer, say 'I don't know.' "
    "Limit the answer to three sentences maximum. Keep it concise. "
    "Also memorize previous conversation context for better future answers."
    "\n\nContext:\n{context}\n"
    "Chat History:\n{chat_history}\n"
    "Question: {question}"
)

In [27]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")

In [28]:
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, chat_prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [29]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue. It occurs when the abnormality develops after bone growth stops, typically in adults. Gigantism, on the other hand, occurs when the abnormality occurs before bone growth stops, resulting in unusual height.


In [30]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know.


In [29]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True,output_key='answer')

C:\Users\klnis\AppData\Local\Temp\ipykernel_18996\3679235723.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True,output_key='answer')


In [ ]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriver,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": chat_prompt},
    return_source_documents=True,
)

In [73]:
def chat_with_bot(user_query):
    score=retriver.vectorstore.similarity_search_with_score(user_query,k=1)
    print(float(score[0][-1])  > 0.60)

    if score[0][-1] > 0.60 :
        response=conversation_chain.invoke({"question":user_query})['answer']
    else:
        response = "I don't know"
    return response


In [74]:
chat_with_bot("what is acne")

True


'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris, the medical term for common acne, is the most common skin disease, affecting nearly 17 million people in the United States.'

In [75]:
chat_with_bot("what treatment i should take for this")

True


"You can consider consulting a dermatologist for personalized advice on treating acne. Previously, you were looking for suggestions on medicines and alternative solutions for acne. I don't know the best treatment for your acne without more information about your condition."

In [76]:
chat_with_bot("what is trigonometry")

False


"I don't know"

In [77]:
chat_with_bot("what is stats")

False


"I don't know"